<cell_type>markdown</cell_type># USGS Earthquakes → Gardner–Knopoff Declustering → California Catalog

This notebook:
1. Downloads earthquakes from the USGS ComCat (FDSN event web service) for a *buffered* bounding box.
2. Applies Gardner–Knopoff (1974) window declustering to label dependent events (foreshocks/aftershocks).
3. Clips to **California** using a state boundary polygon.
4. Saves all earthquakes with `is_mainshock` flag to CSV.
5. Computes and visualizes interarrival times (mainshocks only) for a Poisson-process sanity check.

Key references:
- USGS FDSN Event Web Service parameters (bbox, time, magnitude, etc.).
- van Stiphout et al. (2012) CORSSA review summarizing Gardner–Knopoff windows (Table 2) and discussing **censoring** and why using a buffered region matters.

## 0) Setup

If you don't have the geospatial stack installed, uncomment and run the `pip install` cell.

In [ ]:
# Uncomment if needed
# !pip -q install pandas numpy requests geopandas shapely pyproj pyogrio matplotlib

In [ ]:
import time
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 160)


## 1) Choose your study window + buffered download region

Why buffered? Because declustering depends on events just outside your final region of interest.

For a class demo, a simple buffer that covers CA + NV + neighboring borders is usually fine:
- lon in [-126, -112]
- lat in [31, 43.5]

Then later we clip to California only.


In [ ]:
# Analysis choices
MIN_MAG = 4.0

# Your *analysis* time window (what you'll report results for)
ANALYSIS_START = "1980-01-01"
ANALYSIS_END   = "2025-12-31"

# Add time padding for declustering windows (recommended)
PAD_DAYS_BEFORE = 365   # 1 year
PAD_DAYS_AFTER  = 365   # 1 year

# Buffered bounding box (CA + NV + neighbors)
MIN_LON, MAX_LON = -126.0, -112.0
MIN_LAT, MAX_LAT = 31.0, 43.5

analysis_start_dt = pd.to_datetime(ANALYSIS_START)
analysis_end_dt   = pd.to_datetime(ANALYSIS_END)

download_start_dt = analysis_start_dt - pd.Timedelta(days=PAD_DAYS_BEFORE)
download_end_dt   = analysis_end_dt + pd.Timedelta(days=PAD_DAYS_AFTER)

DOWNLOAD_START = download_start_dt.strftime('%Y-%m-%d')
DOWNLOAD_END   = download_end_dt.strftime('%Y-%m-%d')

DOWNLOAD_START, DOWNLOAD_END

## 2) Download from the USGS event API (GeoJSON)

USGS endpoint (FDSN Event Web Service):
`https://earthquake.usgs.gov/fdsnws/event/1/query`

We'll page using `limit` + `offset`.


In [ ]:
USGS_ENDPOINT = "https://earthquake.usgs.gov/fdsnws/event/1/query"

def fetch_usgs_geojson(
    starttime: str,
    endtime: str,
    minmagnitude: float,
    minlatitude: float,
    maxlatitude: float,
    minlongitude: float,
    maxlongitude: float,
    limit: int = 20000,
    polite_sleep_s: float = 0.5,
    max_pages: int = 1000,
):
    """Fetch events from USGS FDSN Event service using pagination.

    Returns a list of GeoJSON features.
    """
    all_features = []
    offset = 1  # USGS uses 1-based offsets

    for page in range(max_pages):
        params = {
            "format": "geojson",
            "starttime": starttime,
            "endtime": endtime,
            "minmagnitude": minmagnitude,
            "minlatitude": minlatitude,
            "maxlatitude": maxlatitude,
            "minlongitude": minlongitude,
            "maxlongitude": maxlongitude,
            "orderby": "time-asc",
            "limit": limit,
            "offset": offset,
        }

        r = requests.get(USGS_ENDPOINT, params=params, timeout=60)
        if r.status_code == 429:
            # rate-limited
            time.sleep(5)
            continue
        r.raise_for_status()
        data = r.json()
        feats = data.get("features", [])
        if not feats:
            break

        all_features.extend(feats)

        # If we got fewer than 'limit', we're done
        if len(feats) < limit:
            break

        offset += len(feats)
        time.sleep(polite_sleep_s)

    return all_features


def features_to_dataframe(features):
    rows = []
    for f in features:
        prop = f.get("properties", {})
        geom = f.get("geometry", {})
        coords = geom.get("coordinates", [None, None, None])
        lon, lat, depth_km = coords[0], coords[1], coords[2] if len(coords) > 2 else None
        t_ms = prop.get("time")
        t = pd.to_datetime(t_ms, unit="ms", utc=True) if t_ms is not None else pd.NaT
        rows.append({
            "id": f.get("id"),
            "time": t,
            "mag": prop.get("mag"),
            "place": prop.get("place"),
            "type": prop.get("type"),
            "lon": lon,
            "lat": lat,
            "depth_km": depth_km,
        })
    df = pd.DataFrame(rows)
    df = df.dropna(subset=["time", "mag", "lon", "lat"]).copy()
    df["mag"] = df["mag"].astype(float)
    df = df.sort_values("time").reset_index(drop=True)
    return df


features = fetch_usgs_geojson(
    starttime=DOWNLOAD_START,
    endtime=DOWNLOAD_END,
    minmagnitude=MIN_MAG,
    minlatitude=MIN_LAT,
    maxlatitude=MAX_LAT,
    minlongitude=MIN_LON,
    maxlongitude=MAX_LON,
)

df = features_to_dataframe(features)
df.head(), len(df)

## 3) Gardner–Knopoff (1974) windows + declustering

We use the original Gardner & Knopoff (1974) **magnitude-dependent space/time window formulas**:

- **Distance window**: $L = 10^{0.1238 M + 0.983}$ km
- **Time window**: $T = 10^{0.5409 M - 0.547}$ days for $M < 6.5$, or $T = 10^{0.032 M + 2.7389}$ days for $M \geq 6.5$

Implementation approach (similar spirit to OpenQuake's GK declusterer):
- Sort events by **descending magnitude**.
- For each not-yet-assigned event, declare it a potential mainshock.
- Mark any not-yet-assigned events within the GK distance window and within a **time window before/after** as dependent.

We use a symmetric window (foreshocks + aftershocks) by setting `FS_TIME_PROP = 1.0`.

In [ ]:
# Gardner-Knopoff (1974) window formulas
def gk_distance_km(M):
    """GK1974 distance window: L = 10^(0.1238*M + 0.983) km"""
    return 10 ** (0.1238 * M + 0.983)

def gk_time_days(M):
    """GK1974 time window:
    T = 10^(0.5409*M - 0.547) days for M < 6.5
    T = 10^(0.032*M + 2.7389) days for M >= 6.5
    """
    M = np.asarray(M)
    T = np.where(M >= 6.5,
                 10 ** (0.032 * M + 2.7389),
                 10 ** (0.5409 * M - 0.547))
    return T

def gk_windows(mags: np.ndarray):
    """Compute GK distance and time windows for given magnitudes."""
    return gk_distance_km(mags), gk_time_days(mags)


def haversine_km(lon1, lat1, lon2, lat2):
    """Vectorized haversine distance in km."""
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371.0088
    return R * c


def gardner_knopoff_decluster(df_events: pd.DataFrame, fs_time_prop: float = 1.0):
    """Decluster using a GK-style mainshock window method.

    Returns df with extra columns:
      - cluster_id (0 means unclustered/singleton)
      - is_dependent (True if in a cluster but not the main event)
      - is_mainshock (True if representative of its cluster or singleton)
    """
    df0 = df_events.sort_values("time").reset_index(drop=True).copy()
    n = len(df0)

    # Precompute windows per event magnitude
    L_km, T_days = gk_windows(df0["mag"].values)
    df0["L_km"] = L_km
    df0["T_days"] = T_days

    # For speed
    times = df0["time"].values.astype("datetime64[ns]")
    t_days = times.astype("datetime64[ns]").astype("int64") / (1e9 * 86400.0)
    lon = df0["lon"].values
    lat = df0["lat"].values
    mag = df0["mag"].values

    # Work in descending magnitude like OpenQuake (largest event anchors a cluster)
    idx_desc = np.argsort(-mag, kind="mergesort")

    cluster_id = np.zeros(n, dtype=int)
    is_dependent = np.zeros(n, dtype=bool)

    next_cluster = 1
    for ii in idx_desc:
        if cluster_id[ii] != 0:
            continue

        # Candidate mainshock ii: find unassigned events within its time window
        dt = t_days - t_days[ii]
        in_time = (cluster_id == 0) & (dt >= -T_days[ii] * fs_time_prop) & (dt <= T_days[ii])
        if not np.any(in_time):
            continue

        # Now apply distance window among those in time window
        cand = np.where(in_time)[0]
        dkm = haversine_km(lon[cand], lat[cand], lon[ii], lat[ii])
        in_both = cand[dkm <= L_km[ii]]

        # Exclude itself; if nothing else, leave as singleton (cluster_id stays 0)
        in_both_wo_self = in_both[in_both != ii]
        if in_both_wo_self.size == 0:
            continue

        # Assign cluster id to all including mainshock
        cluster_id[in_both] = next_cluster
        # Dependent: everything except mainshock index ii
        is_dependent[in_both_wo_self] = True
        is_dependent[ii] = False
        next_cluster += 1

    df0["cluster_id"] = cluster_id
    df0["is_dependent"] = is_dependent
    df0["is_mainshock"] = ~df0["is_dependent"]

    return df0


df_gk = gardner_knopoff_decluster(df, fs_time_prop=1.0)
df_gk[["time","mag","lat","lon","cluster_id","is_dependent","is_mainshock"]].head(10), df_gk["is_mainshock"].mean()

## 4) Clip to California (state boundary polygon)

We use the U.S. Census Bureau TIGER/Line state boundary shapefile.


In [ ]:
import os
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# TIGER/Line states shapefile (2024) zip
TIGER_URL = "https://www2.census.gov/geo/tiger/TIGER2024/STATE/tl_2024_us_state.zip"
ZIP_PATH = DATA_DIR / "tl_2024_us_state.zip"

if not ZIP_PATH.exists():
    print("Downloading TIGER state boundaries…")
    r = requests.get(TIGER_URL, stream=True, timeout=120)
    r.raise_for_status()
    with open(ZIP_PATH, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)

# Read CA polygon directly from the zip
states = gpd.read_file(f"zip://{ZIP_PATH}")
ca = states[states["NAME"] == "California"].to_crs("EPSG:4326")
ca = ca[["NAME","geometry"]].reset_index(drop=True)
ca

In [ ]:
# Convert events to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_gk,
    geometry=gpd.points_from_xy(df_gk["lon"], df_gk["lat"]),
    crs="EPSG:4326",
)

# Clip ALL earthquakes to CA (keep is_mainshock flag for later filtering)
gdf_ca = gdf[gdf.within(ca.geometry.iloc[0])].copy()

# Restrict to your *analysis* time window
gdf_ca = gdf_ca[(gdf_ca["time"] >= pd.to_datetime(ANALYSIS_START, utc=True)) &
                (gdf_ca["time"] <= pd.to_datetime(ANALYSIS_END, utc=True))].copy()

print("Total downloaded (buffered bbox):", len(df_gk))
print("CA earthquakes (clipped):", len(gdf_ca))
print("  - Mainshocks:", gdf_ca["is_mainshock"].sum())
print("  - Dependent (fore/aftershocks):", (~gdf_ca["is_mainshock"]).sum())

gdf_ca[["time","mag","place","is_mainshock","cluster_id"]].head()

## 5) Interarrival times + quick Poisson sanity checks

For a homogeneous Poisson process, interarrival times are i.i.d. exponential.
Below: histogram + empirical CDF vs fitted exponential, plus an exponential Q-Q plot.


In [ ]:
# Export dataframe (all earthquakes, with is_mainshock flag)
df_ca = pd.DataFrame(gdf_ca.drop(columns="geometry"))
df_ca = df_ca.sort_values("time").reset_index(drop=True)

# For interarrival analysis, filter to mainshocks only
df_mainshocks = df_ca[df_ca["is_mainshock"]].copy().reset_index(drop=True)

# Interarrival times in days (mainshocks only)
t = df_mainshocks["time"].values.astype("datetime64[ns]")
dt_days = (t[1:] - t[:-1]).astype("timedelta64[s]").astype(float) / 86400.0
dt_days = pd.Series(dt_days, name="interarrival_days")

print(f"Mainshocks: {len(df_mainshocks)}")
print(f"Interarrival times: {len(dt_days)}")
dt_days.describe()

In [ ]:
# Fit exponential by MLE: lambda = 1/mean
mean_dt = dt_days.mean()
lam = 1.0 / mean_dt

x = np.linspace(0, np.quantile(dt_days, 0.99), 300)
F_exp = 1 - np.exp(-lam * x)

plt.figure(figsize=(7,4))
plt.hist(dt_days, bins=40, density=True)
plt.plot(x, lam * np.exp(-lam * x), linewidth=2)
plt.xlabel("Interarrival time (days)")
plt.ylabel("Density")
plt.title("Interarrival times: histogram + fitted exponential")
plt.show()

plt.figure(figsize=(7,4))
sorted_dt = np.sort(dt_days.values)
ecdf = np.arange(1, len(sorted_dt)+1) / len(sorted_dt)
plt.plot(sorted_dt, ecdf, label="Empirical CDF")
plt.plot(x, F_exp, linewidth=2, label="Fitted Exp CDF")
plt.xlabel("Interarrival time (days)")
plt.ylabel("CDF")
plt.title("ECDF vs fitted exponential CDF")
plt.legend()
plt.show()


In [ ]:
# Exponential Q-Q plot: compare sorted_dt to theoretical quantiles
p = (np.arange(1, len(sorted_dt)+1) - 0.5) / len(sorted_dt)
q_theory = -np.log(1 - p) / lam

plt.figure(figsize=(5,5))
plt.scatter(q_theory, sorted_dt, s=10)
mx = max(q_theory.max(), sorted_dt.max())
plt.plot([0, mx], [0, mx], linewidth=2)
plt.xlabel("Theoretical Exp quantiles (days)")
plt.ylabel("Empirical quantiles (days)")
plt.title("Exponential Q-Q plot")
plt.show()


## 6) Export (optional)

Save your final declustered, California-only catalog to CSV.


In [ ]:
# Save all CA earthquakes (with is_mainshock flag) to CSV
OUT_PATH = DATA_DIR / "california_earthquakes_declustered.csv"
df_ca.to_csv(OUT_PATH, index=False)
print(f"Saved {len(df_ca)} earthquakes to {OUT_PATH}")
print(f"  - Mainshocks: {df_ca['is_mainshock'].sum()}")
print(f"  - Dependent: {(~df_ca['is_mainshock']).sum()}")